# Amazon Bedrock AgentCore Runtime에서 Bedrock Manus 호스팅하기 가이드

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime을 사용하여 기존 Strands SDK 기반으로 로컬에서 돌아가던 Bedrock Manus 에이전트를 Bedrock AgentCore 환경에 호스팅하는 방법을 배웁니다.


### Overview

시연 목적으로 **1) Strands 에이전트를 로컬에서 호스팅하는 예제**를 먼저 살펴보고, **2) 같은 에이전트를 AgentCore Runtime에 올리는 예제**를 이어서 보며 두 방식을 비교해보도록 하겠습니다.


| 정보                | 세부사항                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| 에이전트 유형        | 멀티 에이전트 (Planner, Supervisor, Coder, Reporter)                                                                         |
| 에이전틱 프레임워크   | Strands                                                          |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                    |
| 튜토리얼 구성요소    | AgentCore Runtime에서 에이전트 호스팅. Strands SDK 및 Amazon Bedrock 모델 사용 |
| 튜토리얼 분야        | 범용                                                                         |
| 예제 복잡도         | 쉬움                                                                         |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK 및 boto3                                 |


### 목차

#### 1) 로컬에서 에이전트 실행/호출하기
AgentCore Runtime 배포 전 로컬 환경에서의 개발 및 테스트 과정

- **Step 1.** 에이전트 코드 작성
- **Step 2.** 로컬 실행 후 테스트

#### 2) AgentCore Runtime에 에이전트 배포 후 호출하기
Amazon Bedrock AgentCore starter toolkit을 활용한 프로덕션 배포 과정

- **Step 1.** AgentCore Runtime용 에이전트 코드 작성
- **Step 2.** 배포 전 설정: agentcore_runtime.configure()
- **Step 3.** 배포: agentcore_runtime.launch()
- **Step 4.** 호출: agentcore_runtime.invoke()


## 사전 준비사항
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Docker running

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

# 1) Local에서 에이전트 실행/호출하기 

AgentCore Runtime에 에이전트를 배포하기 전에, 시연 목적으로 Bedrock Manus를 로컬에서 개발하고 실행하는 과정을 보겠습니다.

- Step 1. 에이전트 코드 작성
- Step 2. 로컬 실행 후 테스트 

### Step 1. 에이전트 코드 작성
먼저 Strands SDK로 작성된 Bedrock Manus 에이전트의 핵심 메인 코드를 살펴보겠습니다.


### Step 2. 로컬 실행 및 테스트

In [10]:
# # 실행 예시:
# !python local_agent.py '{"prompt": "moon market 판매 현황 보고서를 작성해주세요."}'


- - - 
# 2) AgentCore Runtime에 에이전트 배포 후 호출하기 

이제 위 Bedrock Manus를 AgentCore Runtime에 배포해보겠습니다.    

[Amazon Bedrock AgentCore starter toolkit](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-starter-toolkit.html)을 이용해 에이전트를 AgentCore 런타임에 쉽게 배포할 수 있습니다. 아래 4개의 Step을 따라가시면, Bedrock Manus를 AgentCore Runtime에서 테스트해볼 수 있습니다. 

- Step 1. AgentCore Runtime용 에이전트 코드 작성
- Step 2. 배포 전 설정: `agentcore_runtime.configure()`
- Step 3. 배포: `agentcore_runtime.launch`
- Step 4. 호출: `agentcore_runtime.invoke()`


### Step 1. AgentCore Runtime용 에이전트 코드 작성
기존 코드를 AgentCore Runtime 용 에이전트로 변경하는 것은 어렵지 않습니다. 위의 코드에서 아래 4가지만 추가해주면 됩니다:

- `from bedrock_agentcore.runtime import BedrockAgentCoreApp`로 Runtime App 가져오기
- 코드에서 `app = BedrockAgentCoreApp()`로 App 초기화
- 호출할 메인 함수를 `@app.entrypoint` 데코레이터로 장식
- `app.run()`으로 AgentCoreRuntime이 에이전트 실행을 제어하도록 하기


**AgentCore Runtime에 배포할 에이전트가 준비되었습니다!** (agentcore_runtime.py) 

이제 이 에이전트를 클라우드 환경에 배포해볼까요?



### Step 2. IAM Role 설정 및 배포 구성: `agentcore_runtime.configure()`  

먼저 AgentCore Runtime이 사용할 IAM Role을 만들고,    
다음으로 starter toolkit을 사용하여 진입점 (entrypoint)과 생성한 IAM Role, 그리고 패키지 설치를 위한 requirements 파일을 연결하여 AgentCore Runtime 배포를 구성하겠습니다. 

In [11]:
# IAM Role 생성

import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from src.utils.agentcore import create_agentcore_role

agent_name="bedrock_manus_runtime"
# agentcore_iam_role = create_agentcore_role(agent_name=agent_name)
# print(agentcore_iam_role)
# agentcore_iam_role = "arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role"

sys.path[0]: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications


In [12]:
# 올바른 방법 1: 딕셔너리로 직접 할당
agentcore_iam_role = {
    'Role': {
        'Path': '/',
        'RoleName': 'agentcore-bedrock_manus_runtime-role',
        'RoleId': 'AROAQ24BP2I6GBFPY65CS',
        'Arn': 'arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role',
        'CreateDate': '2025-09-28T01:33:47Z',  # datetime 객체 대신 문자열 사용
        'AssumeRolePolicyDocument': {
            'Version': '2012-10-17',
            'Statement': [{
                'Sid': 'AssumeRolePolicy',
                'Effect': 'Allow',
                'Principal': {'Service': 'bedrock-agentcore.amazonaws.com'},
                'Action': 'sts:AssumeRole',
                'Condition': {
                    'StringEquals': {'aws:SourceAccount': '057716757052'},
                    'ArnLike': {'aws:SourceArn': 'arn:aws:bedrock-agentcore:us-east-1:057716757052:*'}
                }
            }]
        }
    },
    'ResponseMetadata': {
        'RequestId': 'fb9f7061-c02f-4b2c-aa63-768fc0d0c9d5',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Sun, 28 Sep 2025 01:33:47 GMT',
            'x-amzn-requestid': 'fb9f7061-c02f-4b2c-aa63-768fc0d0c9d5',
            'content-type': 'text/xml',
            'content-length': '1118'
        },
        'RetryAttempts': 0
    }
}

In [13]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
print("region", region)

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    agent_name=agent_name,
    # entrypoint="runtime.py",
    entrypoint="agentcore_runtime.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,  # starter toolkit이 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 구성
    requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/agentcore_runtime.py, bedrock_agentcore_name=agentcore_runtime
Configuring BedrockAgentCore agent: bedrock_manus_runtime


region us-east-1


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/Dockerfile
Generated .dockerignore: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.dockerignore
Keeping 'bedrock_manus_runtime' as default agent
Bedrock AgentCore configured: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/.dockerignore'), runtime='Docker', region='us-east-1', account_id='057716757052', execution_role='arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role', ecr_repository=None, auto_create_ecr=True)

#### 더 알아보기
**Signature:**
```python
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
```

**Docstring:**  
Configure Bedrock AgentCore from notebook using an entrypoint file.

**Args:**
- **entrypoint**: Path to Python file with optional Bedrock AgentCore name (e.g., "handler.py" or "handler.py:bedrock_agentcore")
- **execution_role**: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
- **agent_name**: name of the agent
- **requirements**: Optional list of requirements to generate requirements.txt
- **requirements_file**: Optional path to existing requirements file
- **ecr_repository**: Optional ECR repository URI
- **container_runtime**: Optional container runtime (docker/podman)
- **auto_create_ecr**: Whether to auto-create ECR repository
- **auto_create_execution_role**: Whether to auto-create execution role (makes execution_role optional)
- **authorizer_configuration**: JWT authorizer configuration dictionary
- **region**: AWS region for deployment
- **protocol**: agent server protocol, must be either HTTP or MCP

### Step 3. 배포: `agentcore_runtime.launch()`
위 configure 명령어를 통해, **에이전트 배포를 위한 Dockerfile이 준비**되었습니다. 

이제 `launch()` 명령어를 이용해서 위 Dockerfile을 기반으로 에이전트의 컨테이너 이미지를 생성해 Amazon ECR 레포지토리에 올린 뒤, AgentCore Runtime에 배포해보겠습니다.


In [16]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'bedrock_manus_runtime' to account 057716757052 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 057716757052.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-bedrock_manus_runtime
Using execution role from config: arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role
Preparing CodeBuild project and uploading source...
Using CodeBuild role from config: arn:aws:iam::057716757052:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-b015ee5cd0
Using .dockerignore with 43 patterns
Uplo

In [17]:
import platform
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

Machine: x86_64
Processor: x86_64
Architecture: ('64bit', 'ELF')
Platform: Linux-6.8.0-1039-aws-x86_64-with-glibc2.35


#### Step 3-a. 배포 완료 후, 잘 배포되었는지 상태 체크: `agentcore_runtime.status()`

In [22]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: bedrock_manus_runtime
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: bedrock_manus_runtime


'READY'

### Step 4. 배포한 에이전트 호출: `invoke_agent_runtime()`

In [13]:
import json
import boto3
from boto3.session import Session
from src.utils.strands_sdk_utils import strands_utils

boto_session = Session()
region = boto_session.region_name

def parse_sse_data(sse_bytes):
    if not sse_bytes or len(sse_bytes) == 0:
        return None

    try:
        text = sse_bytes.decode('utf-8').strip()
        if not text or text == '': return None

        if text.startswith('data: '):
            json_text = text[6:].strip()
            if json_text: return json.loads(json_text)
        else:
            return json.loads(text)

    except Exception as e:
        pass

    return None

agent_arn = launch_result.agent_arn
from botocore.config import Config

my_config = Config(
    connect_timeout=60*100,
    read_timeout=60*5*2, # 10분
)

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    config=my_config,
)

# 복잡한 분석 프롬프트 (주석 처리)
# boto3_response = agentcore_client.invoke_agent_runtime(
#     agentRuntimeArn=agent_arn,
#     qualifier="DEFAULT",
#     payload=json.dumps({"prompt": '이것은 아마존 상품판매 데이터를 분석하고 싶습니다. 분석대상은 "./data/Dat-fresh-food-claude.csv" 파일 입니다. 데이터를 기반으로 마케팅 인사이트 추출을 위한 분석을 진행해 주세요. 분석은 기본적인 데이터 속성 탐색 부터, 상품 판매 트렌드, 변수 관계, 변수 조합 등 다양한 분석 기법을 수행해 주세요. 데이터 분석 후 인사이트 추출에 필요한 사항이 있다면 그를 위한 추가 분석도 수행해 주세요. 분석 리포트는 상세 분석과 그 것을 뒷받침 할 수 있는 이미지 및 차트를 함께 삽입해 주세요. 최종 리포트는 pdf 형태로 저장해 주세요.'})
# )

# 간단한 분석 프롬프트
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "./data/Dat-fresh-food-claude.csv 파일을 분석해서 총 매출액을 계산하고, 카테고리별 매출 비중도 함께 보여줘. 그리고 pdf 로 보고서 생성해줘"})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for event in boto3_response["response"].iter_lines(chunk_size=1):
        event = parse_sse_data(event)
        if event is None:  # None 체크 추가
            continue
        else:
            strands_utils.process_event_for_display(event)
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            print ("6", event)
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

handoff_to_planner: I'll need to consult our planning system for this request.Let me analyze the user's request to develop a comprehensive plan. The user wants me to:

1. Analyze the file `./data/Dat-fresh-food-claude.csv`
2. Calculate the total sales amount (총 매출액)
3. Calculate sales proportion by category (카테고리별 매출 비중)
4. Generate a PDF report with this information

This is a data analysis task requiring numerical calculations (total sales, category proportions) and report generation. Based on the mandatory workflow rules, we need to use Coder for the data analysis, Validator for verifying the calculations, and Reporter for generating the PDF report.

Let me create a detailed plan following the required agent loop structure:

1. **Coder**: Will load the CSV file, perform data analysis, calculate total sales, and determine category-wise sales proportions. Will also create visualizations and generate metadata for validation.
2. **Validator**: Will verify all calculations performed by t

ReadTimeoutError: Read timeout on endpoint URL: "None"